In this tutorial, we look at the [Happy Planet Index](http://happyplanetindex.org/) dataset, which contains metrics related to well-being for 140 countries around the world. We demonstrate how you can globally define custom actions on the lux API, and share them across dataframes.

In [ ]:
import pandas as pd
import lux

The dataframe initially registers a few default recommendations, such as Correlation, Enhance, Filter, etc.

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/lux-org/lux-datasets/master/data/hpi.csv")
lux.config.default_display = "lux"
df

Here, we show an example of how the G10 mean difference can be displayed using a VisList. Now we can look at the attributes that are most different across G10 and non-G10 countries. We find that G10 countries have higher GDPPerCapita, but also contribute to more Footprint.

Following this example, we will integrate this VisList into the list of recommendations using a function to register custom actions.

In [ ]:
df["G10"]  = df["Country"].isin(["Belgium","Canada","France","Germany","Italy","Japan","Netherlands","United Kingdom","Switzerland","Sweden","United States"])

In [ ]:
from lux.vis.VisList import VisList

In [ ]:
intent = [lux.Clause("?",data_type="quantitative"),lux.Clause("G10")]
vlist = VisList(intent,df)

for vis in vlist:
    # Percentage Change Between G10 v.s. non-G10 countries 
    a = vis.data.iloc[0,1]
    b = vis.data.iloc[1,1]
    vis.score = (b-a)/a
vlist = vlist.topK(15)

In [ ]:
vlist

Let's define a custom Callable function to generate the recommendations on the dataframe. In this example, we will use G10 to generate a VisList to calculate the percentage change of means Between G10 v.s. non-G10 countries.

In [ ]:
def G10_mean_difference(input_df): #TODO: change input_df name? 
    intent = [lux.Clause("?",data_type="quantitative"),lux.Clause("G10")]
    vlist = VisList(intent,input_df)

    for vis in vlist:
        a = vis.data.iloc[0,1]
        b = vis.data.iloc[1,1]
        vis.score = (b-a)/a
    vlist = vlist.topK(15)
    return {"action":"G10", "description": "Percentage Change of Means Between G10 v.s. non-G10 countries", "collection": vlist}

In this block, we define a display condition function to determine whether or not we want to generate recommendations for the custom action. In this example, we simply check if we are using the HPI dataset to generate recommendations for G10.

In [ ]:
def is_G10_hpi_dataset(df):
    try: 
        return all(df.columns == ['HPIRank', 'Country', 'SubRegion', 'AverageLifeExpectancy',
       'AverageWellBeing', 'HappyLifeYears', 'Footprint',
       'InequalityOfOutcomes', 'InequalityAdjustedLifeExpectancy',
       'InequalityAdjustedWellbeing', 'HappyPlanetIndex', 'GDPPerCapita',
       'Population', 'G10'])
    except: 
        return False

Here, you can register the provided action globally in lux. The function register_action requires an name (str), action, (optional) display_condition and (optional) additional args for the action as parameters.

In [ ]:
lux.register_action("G10", G10_mean_difference)

We can generate the new custom recomendations by calling the Lux dataframe again.

In [ ]:
df

Using our new action, we can modify our display to only show countries with that reach a certain threshold of GDP and see how their G10 difference compares. 

In [ ]:
df[df["GDPPerCapita"]>40000]

To see what actions are defined on the Lux Action Manager, the following lines allow you to navigate lux.actions to see both default and user-defined actions.

In [ ]:
lux.actions.__len__()

In [ ]:
lux.actions.__getactions__()

You can also get a single action attribute by calling this function with the action's name.

In [ ]:
lux.actions.__getattr__("G10")

This simple function allows you to remove from Lux's action manager an action with its name. The action will no longer display with the Lux dataframe.

In [ ]:
lux.remove_action("G10")

In [ ]:
df